In [1]:
# import dependencies
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 3000)

In [2]:
cali_tox_df = pd.read_csv('cscpopendata.csv')
discont_chem = pd.read_csv(r'Unique_Chem.csv', usecols=['ChemicalName'])
cali_tox_df.sort_values('CompanyName')

,CDPHId,ProductName,CSFId,CSF,CompanyId,CompanyName,BrandName,PrimaryCategoryId,PrimaryCategory,SubCategoryId,SubCategory,CasId,CasNumber,ChemicalId,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount
89909,33489,Sahajan Radiance Ayurvedic Blend Face Serum,NaN,NaN,1222,2355644 Ontario Inc.,Sahajan,90,Skin Care Products,161,Facial Cream,656,13463-67-7,54088,Titanium dioxide,07/05/2018,07/05/2018,NaN,07/05/2018,07/05/2018,NaN,1
45495,16120,Power Stick Deodorant Body Spray,24077.0,Hurricane,754,"A. P. Deauville, LLC",Power Stick,12,Fragrances,16,Toilet Water/Eaux de Toilette,293,140-67-0,24569,Estragole,02/28/2011,02/28/2011,NaN,02/28/2011,02/28/2011,NaN,1
87751,33013,LATTE PER IL CORPO ALLA COLONIA - BODY LOTION,NaN,NaN,1206,ACQUA DI PARMA,ACQUA DI PARMA,90,Skin Care Products,105,Other Skin Care Product,1046,102-71-6,53292,Triethanolamine,05/10/2018,05/10/2018,NaN,05/10/2018,05/10/2018,NaN,1
87750,33012,CREMA ALLA COLONIA - BODY CREAM,NaN,NaN,1206,ACQUA DI PARMA,ACQUA DI PARMA,90,Skin Care Products,105,Other Skin Care Product,1046,102-71-6,53291,Triethanolamine,05/10/2018,05/10/2018,NaN,05/10/2018,05/10/2018,NaN,1
87752,33014,GELSOMINO NOBILE CREMA RADIOSA PER IL CORPO - ...,NaN,NaN,1206,ACQUA DI PARMA,ACQUA DI PARMA,90,Skin Care Products,105,Other Skin Care Product,656,13463-67-7,53293,Titanium dioxide,05/10/2018,05/10/2018,NaN,05/10/2018,05/10/2018,NaN,1
87753,33015,GELSOMINO NOBILE GEL RADIOSO PER IL CORPO - RA...,NaN,NaN,1206,ACQUA DI PARMA,ACQUA DI PARMA,6,Bath Products,159,Body Washes and Soaps,1044,68140-00-1,53294,Cocamide MEA,05/10/2018,05/10/2018,NaN,05/10/2018,05/10/2018,NaN,1
87754,33016,BALSAMO ALLA COLONIA - AFTER SHAVE BALM,NaN,NaN,1206,ACQUA DI PARMA,ACQUA DI PARMA,82,Shaving Products,83,Aftershave Products,1046,102-71-6,53295,Triethanolamine,05/10/2018,05/15/2018,NaN,05/10/2018,05/10/2018,NaN,1
88314,33128,ROSA NOBILE Gel Vellutato per il Bagno - Velv...,NaN,NaN,1206,ACQUA DI PARMA,ACQUA DI PARMA,6,Bath Products,159,Body Washes and Soaps,1044,68140-00-1,53495,Cocamide MEA,05/23/2018,05/23/2018,NaN,05/23/2018,05/23/2018,NaN,1
87439,32898,MAGNOLIA NOBILE Gel sublime per il bagno Subli...,NaN,NaN,1206,ACQUA DI PARMA,ACQUA DI PARMA,6,Bath Products,159,Body Washes and Soaps,1044,68140-00-1,53078,Cocamide MEA,04/24/2018,04/24/2018,NaN,04/24/2018,04/24/2018,NaN,1
88321,33135,COLLEZIONE BARBIERE GEL EMOLLIENTE - SHAVING GEL,NaN,NaN,1206,ACQUA DI PARMA,ACQUA DI PARMA,82,Shaving Products,86,Shaving Cream and other Beard Softeners,1046,102-71-6,53502,Triethanolamine,05/23/2018,05/23/2018,NaN,05/23/2018,05/23/2018,NaN,1


In [3]:
delete_col = ['CDPHId','CSFId', 'CSF', 'CompanyId','PrimaryCategoryId','SubCategoryId', 'CasId', 'CasNumber', 
              'ChemicalId', 'ChemicalCreatedAt','ChemicalUpdatedAt','ChemicalDateRemoved','CompanyName', 'PrimaryCategory', 'SubCategory',
             'InitialDateReported', 'MostRecentDateReported','ChemicalCreatedAt','ChemicalUpdatedAt','ChemicalDateRemoved']

In [4]:
cali_tox_df = cali_tox_df.drop(columns=delete_col)

In [5]:
# populate discontinuted date to 0 if it is not populated
cali_tox_df['DiscontinuedDate'].fillna(0, inplace=True)

In [6]:
# sort by product name
cali_tox_df = cali_tox_df.sort_values('ProductName')

In [7]:
# drop duplicates based on product name
cali_tox_df = cali_tox_df.drop_duplicates()

In [8]:
# drop column that have do not use in the brand
cali_tox_df = cali_tox_df[cali_tox_df.BrandName != '(do not use)']

In [9]:
cali_tox_df = cali_tox_df.groupby(['ProductName', 'BrandName', 'ChemicalCount','DiscontinuedDate']).agg(lambda x: tuple(x)).applymap(list).reset_index()

In [10]:
# update ingredients from list to string
cali_tox_df['ChemicalName'] = cali_tox_df['ChemicalName'].astype(str)
cali_tox_df['ChemicalName'] = cali_tox_df['ChemicalName'].str.extract('\[(.*?)\]')

In [16]:
# if discontinuted is 0 then update discontinuted column with 1
# if discontinuted is popualted then update discontintued column with 0

cali_tox_df['Discontinued'] = cali_tox_df['DiscontinuedDate'].apply(lambda x: '1' if x == 0 else '0')

In [17]:
cali_tox_df.to_csv('CA_Products.csv', index=False)